In [1]:
import pandas as pd
import numpy as np

계층적 색인(Hierarchical indexing)은 축에 대해 다중(둘 이상) 색인 단계를 지정할 수 있다.  
즉, 차원이 높은(고차원) 데이터를 낮은 차원의 형식으로 다룰 수 있게 해주는 기능이다. (고차라면 3차 이상을 말하는 건가?)

이때 인덱스(data.index)는 MultiIndex 타입을 가진다.   
계층적인 색인은 데이터를 재형성하고 피벗 테이블 생성과 같은 그룹 기반의 작업을 할 때 중요하게 사용됨.

In [10]:
data = pd.Series(np.random.randn(10), index=[['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'd', 'd'],[1,2,3,1,2,3,1,2,2,3]])

In [11]:
data

a  1   -0.760419
   2   -2.770703
   3   -0.825583
b  1    1.760585
   2   -1.104339
   3   -0.068594
c  1   -0.235407
   2   -0.548977
d  2    0.525311
   3    0.464205
dtype: float64

In [8]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 1, 2, 0, 1, 1, 2]])

In [13]:
data['b']

1    1.760585
2   -1.104339
3   -0.068594
dtype: float64

In [26]:
data['b':'c']

b  1    1.760585
   2   -1.104339
   3   -0.068594
c  1   -0.235407
   2   -0.548977
dtype: float64

In [25]:
# 상위 계층 인덱스를 부분적 색인으로 접근(partial indexing)
data.loc[['b', 'd']]

b  1    1.760585
   2   -1.104339
   3   -0.068594
d  2    0.525311
   3    0.464205
dtype: float64

In [27]:
# 하위 계층 객체 선택하기
data[:, 2]

a   -2.770703
b   -1.104339
c   -0.548977
d    0.525311
dtype: float64

In [34]:
data.unstack().unstack().unstack().unstack().unstack()

,1,2,3
a,-0.760419,-2.770703,-0.825583
b,1.760585,-1.104339,-0.068594
c,-0.235407,-0.548977,NaN
d,NaN,0.525311,0.464205


In [66]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)), 
            index=[['a', 'a', 'b', 'b'], [1,2,1,2]],
            columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])

In [67]:
frame.index.name = ['Key1', 'Key2']
frame.columns.name = ['state', 'color']

In [68]:
frame['Ohio']['a':'a']

Green  Red
a 1      0    1
  2      3    4

In [69]:
# Columns 에 대해 두 축을 가진다는 말은 칼럼이 MultiIndex 라는 의미이다.
frame.columns

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           codes=[[1, 1, 0], [0, 1, 0]])

In [70]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']], names=['state', 'color'])

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           codes=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])

In [79]:
frame.swaplevel(0, 1)

Ohio     Colorado
    Green Red    Green
1 a     0   1        2
2 a     3   4        5
1 b     6   7        8
2 b     9  10       11

In [80]:
# 책과는 다르게 DataFrame 은 sortlevel 을 지원하지 않는다.
frame.sortlevel(1)

AttributeError: 'DataFrame' object has no attribute 'sortlevel'

In [90]:
# 책의 본문과는 다르게 index 와 columns 에 MultiIndex  를 적용하는 방법은 아래와 같다.
# frame.index.name = ['key1', 'key2'] 와 같은 방식은 더이상 지원하지 않는 것으로 보인다.
frame2 = pd.DataFrame(np.arange(12).reshape((4,3)), 
            index=pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1,2,1,2]], names=['key1', 'key2']),
            columns=pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']], names=['state', 'color']))

In [96]:
frame2

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [91]:
frame2.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [100]:
frame2.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [104]:
frame3 = pd.DataFrame({
    'a': range(7), 'b': range(7,0,-1),'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
    'd': [0,1,2,0,1,2,3]
})

In [105]:
frame3

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [103]:
# set_index 를 통해서 하나 이상의 칼럼을 색인하는 새로운 DataFrame 을 생성한다.
# set_index 에서 drop 옵션은 Default 로 True 이다. 즉, 'c', 'd' 칼럼을 소거된다.
# 이를 방지하기 위해서 drop=False 옵션을 전달한다.
frame3.set_index(['c', 'd'])

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [108]:
frame3.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [110]:
# 또한 인덱스를 초기화(reset)할 수 있는데, 아래와 같이 사용하면 원래 상태로 돌아간다.
frame3.set_index(['c', 'd']).reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1
